In [1]:
import os
import random
import shutil
from PIL import Image, ImageOps
from tqdm.notebook import tqdm


## Dataset preprocessing

In [2]:
# Путь к папке с файлами
directory = 'dataset/'

files = os.listdir(directory)

### Приведём имена файлов к одному виду

In [3]:
def process_and_rename(files, class_prefix):
    
    counter = 1
    
    for file_name in tqdm(files):
        if file_name.startswith(class_prefix):
            new_file_name = f"{class_prefix}_{counter:04d}.jpg"
            os.rename(os.path.join(directory, file_name), os.path.join(directory, new_file_name))

            counter += 1


process_and_rename(files, 'no')
process_and_rename(files, 'r')
process_and_rename(files, 'yes')

print("Файлы успешно обработаны и переименованы.")


  0%|          | 0/1108 [00:00<?, ?it/s]

  0%|          | 0/1108 [00:00<?, ?it/s]

  0%|          | 0/1108 [00:00<?, ?it/s]

Файлы успешно обработаны и переименованы.


### Найдём наименьшие размеры файлов по каждому из измерений и приведем все файлы к одному размеру

In [4]:
min_width = float('inf')
min_height = float('inf')

files = [f for f in os.listdir(directory) if f.endswith('.jpg')]

# Проход по всем файлам для определения наименьшего размера
for file_name in tqdm(files, desc="Определение наименьшего размера изображений"):
    with Image.open(os.path.join(directory, file_name)) as img:
        width, height = img.size
        min_width = min(min_width, width)
        min_height = min(min_height, height)

print(f"Наименьший размер изображения: {min_width}x{min_height}")


Определение наименьшего размера изображений:   0%|          | 0/1108 [00:00<?, ?it/s]

Наименьший размер изображения: 20x136


### Заодно приведём изображения к оттенкам серого

In [5]:
for file_name in tqdm(files, desc="Изменение размеров и преобразование в grayscale"):
    with Image.open(os.path.join(directory, file_name)) as img:
        # Преобразование изображения в оттенки серого
        img_gray = img.convert('L')
        
        # Изменение размера изображения с использованием LANCZOS
        img_resized = img_gray.resize((min_width, min_height), Image.Resampling.LANCZOS)

        img_resized.save(os.path.join(directory, file_name))

print("Все изображения приведены к оттенкам серого, изменены в размере и сохранены.")



Изменение размеров и преобразование в grayscale:   0%|          | 0/1108 [00:00<?, ?it/s]

Все изображения приведены к оттенкам серого, изменены в размере и сохранены.


### Выведем по несколько изображений каждого класса, визуально оценим, что всё ок

In [6]:
from IPython.display import display, HTML

classes = ['no', 'r', 'yes']

num_images_to_show = 20
for class_name in classes:
    class_files = [f for f in files if f.startswith(class_name)]
    
    html_str = f"<h2>Класс: {class_name}</h2>"
    html_str += '<div style="display:flex; flex-direction:row; justify-content:space-between;">'
    for file_name in class_files[:num_images_to_show]:
        img_path = os.path.join(directory, file_name)
        html_str += f'<img src="{img_path}" style="align:auto; margin: 5px"/>'
    html_str += '</div>'
    
    display(HTML(html_str))


### Выведем количество изображений каждого класса

In [7]:
class_counts = {}

for f in files:
    class_name = f.split('_')[0]
    if class_name in class_counts:
        class_counts[class_name] += 1
    else:
        class_counts[class_name] = 1

print("Количество изображений по классам:")
for class_name, count in class_counts.items():
    print(f"Класс '{class_name}': {count} изображений")

Количество изображений по классам:
Класс 'no': 171 изображений
Класс 'r': 128 изображений
Класс 'yes': 809 изображений


### Сбалансируем классы.

Для классов где не хватает изображений - добавим аугментированные изображения, для класса где изображений слишком много - удалим случайную выборку изображений.

In [8]:
random.seed(42)

source_directory = directory
target_directory = 'augmented_dataset'

if not os.path.exists(target_directory):
    os.makedirs(target_directory)

for file_name in os.listdir(source_directory):
    source_path = os.path.join(source_directory, file_name)
    target_path = os.path.join(target_directory, file_name)
    if not os.path.exists(target_path):
        shutil.copy2(source_path, target_path)

target_count = 500 # Целевое количество изображений в каждом классе

def augment_image(image):
    aug_images = []
    aug_images.append(ImageOps.mirror(image))
    aug_images.append(ImageOps.flip(image))
    return aug_images

for class_name in ['no', 'r', 'yes']:
    class_files = [f for f in os.listdir(target_directory) if f.startswith(class_name)]
    num_files_needed = target_count - len(class_files)
    
    if num_files_needed > 0:  # Если нужны дополнительные изображения
        for i in tqdm(range(num_files_needed + 1), desc=f"Augmenting class '{class_name}'"):
            file_to_augment = random.choice(class_files)
            with Image.open(os.path.join(target_directory, file_to_augment)) as img:
                new_img = random.choice(augment_image(img))
                new_file_name = f"{class_name}_{len(class_files)+i:04d}.jpg"
                new_img.save(os.path.join(target_directory, new_file_name))

    elif num_files_needed < 0:
        files_to_remove = random.sample(class_files, len(class_files) - target_count)
        for file_name in tqdm(files_to_remove, desc=f"Undersampling для класса '{class_name}'"):
            os.remove(os.path.join(target_directory, file_name))

Augmenting class 'no':   0%|          | 0/330 [00:00<?, ?it/s]

Augmenting class 'r':   0%|          | 0/373 [00:00<?, ?it/s]

Undersampling для класса 'yes':   0%|          | 0/309 [00:00<?, ?it/s]

### Выведем количество примеров каждого класса после балансировки

In [9]:
aug_files = os.listdir(target_directory)

class_counts = {}

for f in aug_files:
    class_name = f.split('_')[0]
    if class_name in class_counts:
        class_counts[class_name] += 1
    else:
        class_counts[class_name] = 1

print("Количество изображений по классам:")
for class_name, count in class_counts.items():
    print(f"Класс '{class_name}': {count} изображений")

Количество изображений по классам:
Класс 'no': 500 изображений
Класс 'r': 500 изображений
Класс 'yes': 500 изображений


## Датасет предобработан, сбалансирован и готов к использованию для тренировки модели.